### Import stuff and setup

In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [1]:
import time

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

# First setup
conf = SparkConf().setAppName("BDProject").setMaster("local")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

### Logistic Regression

In [2]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

def TrainLR(trainingData,testData):
    # Train a  model.
    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
    model = lr.fit(trainingData)
    # Make predictions.
    predictions = model.transform(testData)
    
    # Select (prediction, true label) and compute test error
    evaluator = BinaryClassificationEvaluator(
        labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
    auc = evaluator.evaluate(predictions)
    print("AUC = %g" % (auc,))
    
    return model

### SVM

In [3]:
from pyspark.ml.classification import LinearSVC

def TrainSVM(trainingData,testData):    
    # create the trainer and set its parameters
    lsvc = LinearSVC(maxIter=10, regParam=0.1)

    # train the model
    model = lsvc.fit(trainingData)

    # Make predictions.
    predictions = model.transform(testData)

    # Select (prediction, true label) and compute test error
    evaluator = BinaryClassificationEvaluator(
        labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
    auc = evaluator.evaluate(predictions)
    print("AUC = %g" % (auc,))
    
    return model

### Read and prepare data, run models

In [4]:
from utils import *

HIGGS

In [5]:
# Load and parse the data file, converting it to a DataFrame.
higgs = sqlContext.read.load('Datasets/HIGGS/HIGGS.csv', 
                          format='com.databricks.spark.csv', 
                          header='false', 
                          inferSchema='true')

In [6]:
higgs.head()

Row(_c0=1.0, _c1=0.869293212890625, _c2=-0.6350818276405334, _c3=0.22569026052951813, _c4=0.327470064163208, _c5=-0.6899932026863098, _c6=0.7542022466659546, _c7=-0.24857313930988312, _c8=-1.0920639038085938, _c9=0.0, _c10=1.3749921321868896, _c11=-0.6536741852760315, _c12=0.9303491115570068, _c13=1.1074360609054565, _c14=1.138904333114624, _c15=-1.5781983137130737, _c16=-1.046985387802124, _c17=0.0, _c18=0.657929539680481, _c19=-0.010454569943249226, _c20=-0.0457671694457531, _c21=3.101961374282837, _c22=1.353760004043579, _c23=0.9795631170272827, _c24=0.978076159954071, _c25=0.9200048446655273, _c26=0.7216574549674988, _c27=0.9887509346008301, _c28=0.8766783475875854)

In [15]:
higgs[0]

Column<b'_c0'>

In [7]:
labelcol = '_c0'
readycols = higgs.columns[1:]
dataset = higgs
nclasses = 2
dataset_name = 'higgs'

trainingData,testData = Prepare(dataset,labelcol,readycols,[])

In [24]:
start = time. time()
model = TrainLR(trainingData,testData)
end = time. time()
print(end - start)
#model.save('Models/'+dataset_name+'_LR')

AUC = 0.5
1135.1693818569183


In [ ]:
model = TrainSVM(trainingData,testData)
#model.save('Models/'+dataset_name+'_SVM')

HEPMASS

In [5]:
# Load and parse the data file, converting it to a DataFrame.
hepmass = sqlContext.read.load('Datasets/HEPMASS/all_train.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          delimiter=',',
                          inferSchema='true')

In [6]:
labelcol = '# label'
readycols = hepmass.columns[1:]
dataset = hepmass
nclasses = 2
dataset_name = 'hepmass'

trainingData,testData = Prepare(dataset,labelcol,readycols,[])

In [7]:
model = TrainLR(trainingData,testData)
#model.save('Models/'+dataset_name+'_RF')

AUC = 0.800718


TypeError: cannot unpack non-iterable LogisticRegressionModel object

In [9]:
model = TrainSVM(trainingData,testData)
#model.save('Models/'+dataset_name+'_MLP')

AUC = 0.823273
